In [20]:
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import bandit.main as bd

In [21]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [22]:
def save_excel(df,col):
    dfgb = df.groupby([col])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    # dfgb.to_excel('AllProjectList.xlsx')
    return dfgb

In [23]:
subprocess.Popen('ProjectTime.xlsx', shell=True)

In [24]:
df = get_frame('ProjectTime.xlsx')

In [25]:
# reduce it
df = bd.reduce(df,'Project')
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
df_quality = bd.get_quality(df['Duration'])
df = df.join(df_quality)

32 0.92 29


In [26]:
df.tail(10)

,Project,ProjeDate,Duration,Quality
31,FlaskMegaTut,2018-09-08,4154.0,1.000000
32,FlaskMegaTut,2018-09-08,3908.0,0.938098
33,FlaskMegaTut,2018-09-11,1919.0,0.437594
34,FlaskMegaTut,2018-09-12,2193.0,0.506543
35,FlaskMegaTut,2018-09-18,1581.0,0.352542
36,FlaskMegaTut,2018-09-21,1895.0,0.431555
37,VueClass,2018-09-28,897.0,0.180423
38,FlaskMegaTut,2018-10-05,3000.0,0.709612
39,Octoparse,2018-10-07,1548.0,0.344238
40,Octoparse,2018-10-11,180.0,0.000000


In [27]:
df.sort_values('Quality', ascending=False).head(10)

,Project,ProjeDate,Duration,Quality
31,FlaskMegaTut,2018-09-08,4154.0,1.000000
32,FlaskMegaTut,2018-09-08,3908.0,0.938098
23,SpotifyPlayGen,2018-08-04,3260.0,0.775038
38,FlaskMegaTut,2018-10-05,3000.0,0.709612
30,SpotifyPlayGen,2018-09-03,2894.0,0.682939
20,GoogleCrashCourse,2018-07-29,2880.0,0.679416
17,ZeroToDeepLearn,2018-07-22,2748.0,0.646200
16,ZeroToDeepLearn,2018-07-20,2620.0,0.613991
22,SpotifyPlayGen,2018-08-02,2584.0,0.604932
18,GoogleCrashCourse,2018-07-24,2520.0,0.588827


In [28]:
dfgb = save_excel(df,'Project')

In [29]:
df = df.merge(dfgb)
dfgb['total'] = dfgb['count'].sum()
dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1)
dfgb.sort_values('donext', ascending=False)

,Project,count,mean,var,total,donext
6,TrollSniffer,2,0.437720,0.000448,38,1.112033
2,FlaskMegaTut,7,0.625135,0.067770,38,0.985570
5,SpotifyPlayGen,6,0.583837,0.016294,38,0.973152
0,CUDAproject,4,0.482763,0.001995,38,0.959574
3,GoogleCrashCourse,4,0.468357,0.043491,38,0.945168
8,ZeroToDeepLearn,6,0.530238,0.013843,38,0.919553
1,FindYourJoy,4,0.417401,0.001990,38,0.894212
4,Octoparse,2,0.172119,0.059250,38,0.846432
7,VueClass,3,0.281329,0.007716,38,0.831903
